In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

---
<center><h1> Chapter 5: The Wave Equation </h1></center>

---


---
## Introduction
---

The wave equation is a type of hyperbolic PDE (by comparison, the heat equation was a type of parabolic PDE). 
The general form in a single spatial dimension is

$$
    u_{tt} = c^2 u_{xx}.
$$

The parameter $c$ is called the ***wave speed***, and we will typically just set it to be $1$ for simplicity. 

As before, we will specify boundary conditions on the spatial domain.
However, unlike before, the wave equation involves two derivatives in time, so initial conditions are typically specified for both $u(x,0)$ and $u_t(x,0)$, i.e., we usually specify an initial configuration for the wave $u(x,0)$ as well as the waves initial velocity $u_t(x,0)$.

Therefore, a typical IBVP for the wave equation, using homogeneous Dirichlet conditions on the boundary, is written as

\begin{eqnarray*}
    u_{tt} &=& u_{xx}, \ \ \ \ \ \ \ x\in(0,1), \ t>0, \\
    u(0,t) &=& u(1,t) = 0, \ \ \ \ \ \ \ \ \ \ \ \ \ t>0, \\
    u(x,0) &=& f(x), \ \ \ \ \ x\in(0,1) \\
    u_t(x,0) &=& g(x), \ \ \ \ \ x\in(0,1).
\end{eqnarray*}

---
## Section 5.1: Separation of Variables
---

We play the same game as we did with the heat equation.
   
- Assume that $u(x,t) = X(x)T(t)$

- Insert $X(x)T(t)$ into the wave equation and separate variables

- Solve the same two-point BVP we did before for to get that
<br>
  $$
    X_k(x)=\sin(k\pi x)
  $$ 
<br>
for each eigenvalue $\lambda_k = (k\pi)^2$ for $k\in\mathbb{N}$.
<br><br>
- The IVP for the ODE defined by $T(t)$ is now different with general solutions given by
<br>
  $$
    T_k(t) = a_k \cos(k\pi t) + b_k\sin(k\pi t)
  $$ 
<br><br>
for given eigenvalue $\lambda_k$.
<br><br>
- Conclude that a family of particular solutions is of the form (5.11)
<br><br>
  $$
    u_k(x,t) = \sin(k\pi x)\left(a_k\cos(k\pi t) + b_k\sin(k\pi t)\right)
  $$
<br>   
and that these have initial conditions given by
<br><br>
  $$
   u_k(x,0) = a_k\sin(k\pi x), \ \text{ and } \ (u_k)_t(x,0) = k\pi b_k \sin(k\pi x).
  $$

### Take home message 1
Any linear combination of particular solutions solves the PDE, boundary conditions, and has an initial condition given by the linear combination of the initial conditions for the particular solutions.

### Take home message 2
If initial conditions $f(x)$ and $g(x)$ can be written as Fourier sine series, e.g., as
<br><br>
$$
    f(x) = \sum_{k=1}^\infty a_k\sin(k\pi x) \ \text{ and } \ g(x) = \sum_{k=1}^\infty b_k\sin(k\pi x), 
$$
<br><br>
then the formal solution of the IBVP is given by
<br><br>
$$
    u(x,t) = \sum_{k=1}^\infty \sin(k\pi x)\left(a_k\cos(k\pi t) + \frac{b_k}{k\pi}\sin(k\pi t)\right).
$$


In [ ]:
# Example 5.1

a_1 = 2
b_2 = -1/(2*np.pi)

def u(x,t):
    return a_1*np.sin(np.pi*x)*np.cos(np.pi*t) - \
        b_2 *np.sin(2*np.pi*x)*np.sin(2*np.pi*t)

In [ ]:
%matplotlib widget

    
x = np.linspace(0,1,40)
t = np.linspace(0,6,240)
plt.figure(num=0)
plt.plot(t,u(x[20],t))

x, t = np.meshgrid(x, t)

fig = plt.figure(num=1)
ax = fig.gca(projection='3d')

ax.plot_wireframe(x,t,u(x,t), rstride=20, cstride=40)

---
## Section 5.2: Uniqueness and Energy Arguments
---

The main difference from before is that we now define the "energy" of the system $E(t)$ as
<br><br>
$$
    E(t) = \int_0^1 (u_x^2(x,t) + u_t^2(x,t))\, dx.
$$
<br><br>
We observe that 
<br><br>
$$
    E(0) = \int_0^1 ((f')^2(x,t) + g^2(x,t))\, dx. 
$$
<br><br>
Through some assumptions, Proposition 3.1, and calculus, we get that
<br><br>
$$
    E'(t) = 0 \ \forall t \Rightarrow E(t) = E(0),  \ t\geq 0.
$$
<br><br>
From this, we obtain a stability estimate (Eq. (5.18) in the text) and uniqueness of solutions. 

---
## Section 5.3: A Finite Difference Approximation
---

The text uses centered second order finite difference approximations in both space and time.

In the text, they are deriving an explicit scheme by assuming that the spatial discretization is evaluated at the current time step while in Exercises 5.9 and 5.10 we consider an implicit scheme by instead assuming that the spatial discretization is evaluated at the next time step. 

If we approach this instead from the perspective of doing a semi-discrete discretization, then we arrive at
<br><br>
$$
    v_{tt} = -Av
$$

where $A$ is our usual $n\times n$ matrix from Chapter 2 (the one including the $1/(\Delta x)^2$ factor) that involved discretizing $-u_{xx}$ with a centered finite difference approximation (and since we have $u_{xx}$ in this case, we incur the usual negative sign that we are familiar with from Chapter 4). 
Of course, $v$ is the $n$-dimensional vector that approximates $u$ at the interior grid points (interior because of the Dirichlet boundary conditions).

Unlike our previous studies of the heat equation, we now have $v_{tt}$ instead of $v_t$. 
We use a centered finite difference approximation in time to approximate $v_{tt}$ at a grid point $x_j$ as
<br><br>
$$
    v_{tt}(x_j) \approx \frac{v_j^{m-1} - 2v_{j}^m + v_j^{m+1}}{(\Delta t)^2}.
$$

The ultimate question then becomes whether or not we define the fully discrete problem by setting this equal to the $j$th row of $-Av^{m}$ (for an explicit method) or the $j$th row of $-Av^{m+1}$ (for an implicit method).
Alternatively, we can write everything in matrix-vector format. 

For the explicit method, we have
<br><br>
$$
   \frac{v^{m-1} - 2v^m + v^{m+1}}{(\Delta t)^2} = -Av^m 
$$
<br><br>
and we solve for $v^{m+1}$ to get
<br><br>
$$
    v^{m+1} = (2I-(\Delta t)^2A)v^m - v^{m-1}. 
$$

A detail we have neglected is that applying this equation requires us to use both $v^m$ and $v^{m-1}$.
Thus, we cannot apply this at $m=0$ (which would imply we are solving for $v^1$). 
Somehow, we must specify $v^1$. 
This is done using a Taylor expansion in time.
Specifically, 
<br><br>
$$
    u(x,\Delta t) = u(x,0) + \Delta t u_t(x,0) + \frac{(\Delta t)^2}{2} u_{tt}(x,0) + O((\Delta t)^3), 
$$
<br><br>
and we make the substitutions $u(x,0)=f(x)$, $u_t(x,0) = g(x)$, and $u_{tt}(x,0) = u_{xx}(x,0)$ to get
<br><br>
$$
    u(x,\Delta t) = f(x) + \Delta t g(x) + \frac{(\Delta t)^2}{2} u_{xx}(x,0) + O((\Delta t)^3). 
$$

We therefore specify $v^1 \approx u(x,\Delta t)$ using the initial data as
<br><br>
$$
    v^1 = v^0 + \Delta t \left[\begin{array}{l} g(x_1) \\ g(x_2) \\ \vdots \\ g(x_n) \end{array}\right] - \frac{(\Delta t)^2}{2}Av^0.
$$

In [ ]:
def f(x):
    return x*(1-x)

def g(x):
    return 0*x

In [ ]:
def make_A(n):
    A = np.zeros((n,n))
    np.fill_diagonal(A,2)
    A += np.diag(-np.ones(n-1),k=1)
    A += np.diag(-np.ones(n-1),k=-1)    
    A *= 1/dx**2
    return A

In [ ]:
# Define and plot results for an explicit method

%matplotlib widget

n_s = [19, 39, 99, 199]
errors = np.zeros(4)
dt_s = np.zeros(4)
dx_s = np.zeros(4)
count = 0

for n in n_s:
    
    plt.figure(count)
    
    x = np.linspace(0,1,n+2)
    dx = x[1]-x[0]
    dx_s[count] = dx
    
    A = make_A(n)

    r = 1.0
    dt = r*dx
    dt_s[count] = dt
    
    v_old = f(x)
    v_current = 0*x
    v_current[1:-1] = v_old[1:-1] + dt*g(x[1:-1]) - dt**2/2.*np.dot(A,v_old[1:-1])
    
    v_new = 0*x
    
    t = 1*dt  # <----- start at 1*dt

    # bdry_force = np.zeros(n)

    B = 2*np.eye(n)-dt**2*A
    
    time_iter = 0
    while t<=10:
        v_new[1:-1] = np.dot(B,v_current[1:-1]) - v_old[1:-1]  # <--------- v_{m+1} is now computed
        v_old = np.copy(v_current)
        v_current = np.copy(v_new)
        t += dt  # <-------------- now go to t_{m+1}
        time_iter += 1
        
        if time_iter%5 == 0:
            plt.plot(x, v_new)
    
    plt.plot(x, v_new)
            
    count += 1

### Making movies by exploiting pass-by-reference and in-place operators

In [ ]:
def f(x):  # It is kind of fun to change this
    # Try x*(1-x)
    # Try x*(1-x)*(0.2-x)
    # Try np.sin(k\pi x) for different k
    # Try x*np.sin(1/x)*(1-x)
    return x*(1-x)*(0.2-x)
    

In [ ]:
n = 39  # Choose an n for discretizing space (and subsequently time)

# Now setup the spatial-temporal discretization
x = np.linspace(0,1,n+2)
dx = x[1]-x[0]
A = make_A(n)

r = 1.0
dt = r*dx

B = 2*np.eye(n)-dt**2*A

In [ ]:
# Now setup the numpy arrays that we utilize for visualization purposes
v_old = f(x)
v_current = 0*x
v_current[1:-1] = v_old[1:-1] + dt*g(x[1:-1]) - dt**2/2.*np.dot(A,v_old[1:-1])

def v_at_frame(v_current, v_old):
    # Create the solution to plot in a given frame of the movie
    v_new = np.dot(B,v_current[1:-1]) - v_old[1:-1] 
    
    # Update state of wave in-place
    v_old *= 0
    v_old += v_current
    v_current *= 0
    v_current[1:-1] += v_new
    return

In [ ]:
from matplotlib.animation import FuncAnimation

from IPython.display import HTML

In [ ]:
fig, ax = plt.subplots()
line1 = ax.plot([], [], 'b')[0]  # Using b-. or b: for line-style will create a weird visual
line2 = ax.plot([], [], 'r:')[0]
line3 = ax.plot([], [], markersize=10, marker='s', mfc='k')[0]

ax.set_xlim(0, 1)
ax.set_ylim(-1.1*np.max(np.abs(f(x[1:-1]))), 1.1*np.max(np.abs(f(x[1:-1]))))

plt.title('The Wave')
time_text = ax.text(0.1, 0.9*np.max(np.abs(f(x[1:-1]))), "", 
                    fontsize=15, color='red',
                    bbox=dict(facecolor='blue', alpha=0.1))
plt.close()

def animate_v(frame, v_current, v_old):
    v_at_frame(v_current, v_old)
    line1.set_data((x, v_current))
    t = frame*dt
    time_text.set_text("Time: {:5.2f}".format(t))
    line2.set_data([0.5, 0.5], [-1, 1])
    marker_idx = np.argmax(np.abs(v_current))
    line3.set_data(x[marker_idx],v_current[marker_idx])
    return line1, line2, line3

anim = FuncAnimation(fig, animate_v, frames=int((4/dt)), fargs=(v_current, v_old,), interval=1000*dt)

In [ ]:
HTML(anim.to_jshtml())